<a href="https://colab.research.google.com/github/indranildchandra/rag-done-right-in-production/blob/main/rag_done_right_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Done Right in Production
### Demo Notebook — Zerodha Support FAQ Corpus

**Talk:** Can you RAG like a Pro? · Indranil Chandra, ML & Data Architect, Upstox

---

This notebook is a **self-contained production RAG demo** that runs entirely inside Google Colab — no external API keys required. It builds a real retrieval pipeline on top of Zerodha's public support FAQ corpus.

**What this covers, in order:**

| # | Section | What you will observe |
|---|---------|----------------------|
| 1 | **Corpus Ingestion** | Scrape + parse Zerodha support FAQs |
| 2 | **Chunking** | Semantic chunking vs naive fixed-size |
| 3 | **Qdrant (in-memory)** | Embed + index with `all-MiniLM-L6-v2` |
| 4 | **Hybrid Search** | BM25 + dense retrieval merged with RRF |
| 5 | **Cross-Encoder Reranking** | Two-stage pipeline: recall then precision |
| 6 | **Adaptive-k** | Score-gap boundary vs fixed top-k |
| 7 | **Generation + Citations** | LLM answer grounded with source attribution |
| 8 | **Evaluation** | Hit rate, Faithfulness, Hallucination detection |

> **Runtime:** GPU not required. CPU is sufficient. Estimated full run: ~8–12 minutes.


---
## 0 · Install Dependencies

In [ ]:
%%capture
!pip install qdrant-client sentence-transformers rank-bm25 \
             beautifulsoup4 requests transformers \
             openai tiktoken tqdm colorama --quiet

In [ ]:
import os, re, json, time, random, hashlib, warnings
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from tqdm.auto import tqdm
import requests
from bs4 import BeautifulSoup
import numpy as np

from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance, VectorParams, PointStruct,
    Filter, FieldCondition, MatchValue
)

warnings.filterwarnings('ignore')

# ── Colour helpers for readable notebook output ────────────────────
from colorama import Fore, Style, init as colorama_init
colorama_init(autoreset=True)

def hdr(msg):  print(f"\n{Fore.CYAN}{Style.BRIGHT}{'─'*60}\n  {msg}\n{'─'*60}{Style.RESET_ALL}")
def ok(msg):   print(f"{Fore.GREEN}✓  {msg}{Style.RESET_ALL}")
def info(msg): print(f"{Fore.YELLOW}ℹ  {msg}{Style.RESET_ALL}")
def err(msg):  print(f"{Fore.RED}✗  {msg}{Style.RESET_ALL}")

ok("Imports complete.")

---
## 1 · Corpus Ingestion — Zerodha Support FAQ Scraper

The scraper works in **three passes**:

1. **Category discovery** — read the homepage sidebar to find all section URLs  
2. **Article link extraction** — for each category page, collect article hrefs  
3. **Article fetch + parse** — extract `<h1>` title and body text, strip nav/footer boilerplate  

We scrape politely: 0.4s delay between requests, max 120 articles, 3 retries with backoff.

In [ ]:
# ── Scraper configuration ─────────────────────────────────────────
BASE_URL      = "https://support.zerodha.com"
MAX_ARTICLES  = 120       # cap for demo — raise to 500+ for production
DELAY_SEC     = 0.4       # polite crawl delay
RETRIES       = 3

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (compatible; RAGDemoBot/1.0; "
        "+https://github.com/indranilchandra/rag-demo)"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

# Category sections to crawl — covers trading, account, funds, coin
SEED_CATEGORIES = [
    "/category/trading-and-markets/trading-faqs",
    "/category/trading-and-markets/margins",
    "/category/trading-and-markets/charts-and-orders",
    "/category/trading-and-markets/general-kite",
    "/category/your-zerodha-account/your-profile",
    "/category/your-zerodha-account/account-modification-and-segment-addition",
    "/category/funds/adding-funds",
    "/category/funds/fund-withdrawal",
    "/category/mutual-funds/understanding-mutual-funds",
    "/category/mutual-funds/payments-and-orders",
]

In [ ]:
@dataclass
class ZerodhaArticle:
    url:      str
    title:    str
    body:     str
    category: str
    doc_id:   str = field(init=False)

    def __post_init__(self):
        self.doc_id = hashlib.md5(self.url.encode()).hexdigest()[:12]


def _get(url: str, retries: int = RETRIES) -> Optional[BeautifulSoup]:
    """Fetch a URL and return a BeautifulSoup object. Returns None on failure."""
    full = url if url.startswith("http") else BASE_URL + url
    for attempt in range(retries):
        try:
            r = requests.get(full, headers=HEADERS, timeout=15)
            if r.status_code == 200:
                time.sleep(DELAY_SEC)
                return BeautifulSoup(r.text, "html.parser")
            elif r.status_code == 429:
                wait = 2 ** (attempt + 1)
                info(f"Rate limited — waiting {wait}s")
                time.sleep(wait)
        except requests.RequestException as e:
            time.sleep(2 ** attempt)
    return None


def _extract_article_links(category_url: str) -> List[str]:
    """
    From a category page, collect all /articles/ hrefs.
    Zerodha renders article links as <a href="/category/.../articles/slug">
    """
    soup = _get(category_url)
    if not soup:
        return []
    links = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/articles/" in href and href.startswith("/category/"):
            links.add(href)
    return list(links)


# Boilerplate patterns to strip from article body
_BOILERPLATE = re.compile(
    r"(Updates|Education|Utilities|Support Portal|Related articles|Quick links"
    r"|Signup|About|Products|Pricing|Zerodha Broking|© 20\d\d)",
    re.IGNORECASE
)


def _parse_article(href: str, category: str) -> Optional[ZerodhaArticle]:
    """
    Fetch and parse a single article page.
    Strategy:
      - Title  → <h1> tag
      - Body   → all <p> tags after the <h1>, before 'Related articles'
    """
    soup = _get(href)
    if not soup:
        return None

    h1 = soup.find("h1")
    if not h1:
        return None
    title = h1.get_text(strip=True)

    # Collect paragraphs that appear after the <h1>
    paragraphs = []
    in_body = False
    for tag in soup.find_all(["h1", "h2", "h3", "p", "li"]):
        if tag == h1:
            in_body = True
            continue
        if not in_body:
            continue
        text = tag.get_text(" ", strip=True)
        if not text or len(text) < 15:
            continue
        # Stop at 'Related articles' section
        if re.search(r"related articles", text, re.IGNORECASE):
            break
        if _BOILERPLATE.search(text):
            continue
        paragraphs.append(text)

    body = " ".join(paragraphs).strip()
    if len(body) < 80:   # skip stubs
        return None

    return ZerodhaArticle(
        url=BASE_URL + href,
        title=title,
        body=body,
        category=category,
    )


ok("Scraper functions defined.")

In [ ]:
hdr("Scraping Zerodha Support FAQs")

all_links: Dict[str, str] = {}   # href → category label

for cat_url in SEED_CATEGORIES:
    label = cat_url.split("/")[-1].replace("-", " ").title()
    links = _extract_article_links(cat_url)
    for lnk in links:
        all_links[lnk] = label
    info(f"{label}: {len(links)} article links found")

# Shuffle so we get a diverse sample if we hit the cap
all_link_items = list(all_links.items())
random.seed(42)
random.shuffle(all_link_items)
all_link_items = all_link_items[:MAX_ARTICLES]

ok(f"Total unique article links to fetch: {len(all_link_items)}")

In [ ]:
articles: List[ZerodhaArticle] = []

for href, category in tqdm(all_link_items, desc="Fetching articles"):
    art = _parse_article(href, category)
    if art:
        articles.append(art)

ok(f"Successfully parsed {len(articles)} articles")

# Quick sanity check — show 3 samples
print()
for art in articles[:3]:
    print(f"  [{art.category}]  {art.title}")
    print(f"  Body ({len(art.body)} chars): {art.body[:120]}...")
    print(f"  URL: {art.url}")
    print()

In [ ]:
# Persist to disk — so you can reload without re-scraping
corpus_path = "/content/zerodha_faqs.json"
with open(corpus_path, "w") as f:
    json.dump(
        [{"doc_id": a.doc_id, "url": a.url, "title": a.title,
          "body": a.body, "category": a.category} for a in articles],
        f, indent=2
    )
ok(f"Corpus saved → {corpus_path}  ({os.path.getsize(corpus_path)//1024} KB)")

# Category distribution
from collections import Counter
dist = Counter(a.category for a in articles)
print("\nCategory distribution:")
for cat, cnt in dist.most_common():
    bar = "█" * cnt
    print(f"  {cat:<50} {cnt:>3}  {bar}")

---
## 2 · Chunking Strategy

**The problem with fixed-size chunking:** A 512-token window that splits mid-sentence loses the semantic unit. Two adjacent chunks end up containing half a thought each — both retrieve poorly.

**What we use here:** Sentence-boundary aware chunking with a configurable overlap. Each chunk respects sentence boundaries, with a sliding window overlap to preserve cross-sentence context.

For a support FAQ corpus specifically, most articles are short enough (200–600 tokens) that we treat each article as 1–3 chunks max. The overlap handles the edge case where a key fact straddles a sentence boundary.

**Production note:** For longer documents (regulatory PDFs, annual reports), switch to hierarchical chunking: paragraph-level for indexing, sentence-level for reranking.

In [ ]:
@dataclass
class Chunk:
    chunk_id:   str
    doc_id:     str
    title:      str
    text:       str          # chunk text that gets embedded
    full_text:  str          # full article body — for citation display
    url:        str
    category:   str
    chunk_idx:  int


def sentence_aware_chunk(
    article: ZerodhaArticle,
    max_chars: int = 800,
    overlap_chars: int = 120,
) -> List[Chunk]:
    """
    Split article body into overlapping sentence-boundary-aware chunks.
    Each chunk prepends the article title for embedding context.
    """
    # Split on sentence boundaries
    sentences = re.split(r'(?<=[.!?])\s+', article.body.strip())
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]

    chunks = []
    buf, buf_len = [], 0
    idx = 0

    for sent in sentences:
        if buf_len + len(sent) > max_chars and buf:
            text = " ".join(buf)
            chunks.append(Chunk(
                chunk_id  = f"{article.doc_id}_{idx}",
                doc_id    = article.doc_id,
                title     = article.title,
                text      = f"{article.title}. {text}",
                full_text = article.body,
                url       = article.url,
                category  = article.category,
                chunk_idx = idx,
            ))
            idx += 1
            # Carry over tail sentences for overlap
            overlap_buf, overlap_len = [], 0
            for s in reversed(buf):
                if overlap_len + len(s) < overlap_chars:
                    overlap_buf.insert(0, s)
                    overlap_len += len(s)
                else:
                    break
            buf, buf_len = overlap_buf, overlap_len

        buf.append(sent)
        buf_len += len(sent)

    # Flush remainder
    if buf:
        text = " ".join(buf)
        chunks.append(Chunk(
            chunk_id  = f"{article.doc_id}_{idx}",
            doc_id    = article.doc_id,
            title     = article.title,
            text      = f"{article.title}. {text}",
            full_text = article.body,
            url       = article.url,
            category  = article.category,
            chunk_idx = idx,
        ))

    return chunks


# Build chunk corpus
all_chunks: List[Chunk] = []
for art in articles:
    all_chunks.extend(sentence_aware_chunk(art))

ok(f"{len(articles)} articles → {len(all_chunks)} chunks")
avg_len = sum(len(c.text) for c in all_chunks) / len(all_chunks)
info(f"Average chunk length: {avg_len:.0f} chars")

# Show chunking on a sample article
sample_art = max(articles, key=lambda a: len(a.body))
sample_chunks = sentence_aware_chunk(sample_art)
print(f"\nSample article: '{sample_art.title}'")
print(f"Body length: {len(sample_art.body)} chars → {len(sample_chunks)} chunks")
for i, c in enumerate(sample_chunks):
    print(f"  Chunk {i}: {len(c.text)} chars | '{c.text[:80]}...'")

---
## 3 · Qdrant In-Memory Vector Store

We use **Qdrant's in-memory mode** — no Docker, no external service, no API key. Same Python client API as production Qdrant Cloud. This means your demo code is identical to what you would run in prod — just swap `QdrantClient(':memory:')` for `QdrantClient(url='https://your-cluster.qdrant.io', api_key='...')`.

**Embedding model:** `all-MiniLM-L6-v2`  
- 384 dimensions, 22M parameters, ~80ms per 100 chunks on CPU  
- Production note: for a finance domain, fine-tuned models on SEBI/NSE corpus improve retrieval recall by 10–15%  
- 768-dim models (MPNet) give diminishing returns past this corpus size

In [ ]:
hdr("Loading embedding model")
EMBED_MODEL_NAME = "all-MiniLM-L6-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)
EMBED_DIM = embed_model.get_sentence_embedding_dimension()
ok(f"Model: {EMBED_MODEL_NAME} | Dim: {EMBED_DIM}")

In [ ]:
hdr("Setting up Qdrant in-memory + indexing chunks")

COLLECTION = "zerodha_faqs"

# In-memory Qdrant — identical API to Qdrant Cloud
qdrant = QdrantClient(":memory:")
qdrant.create_collection(
    collection_name=COLLECTION,
    vectors_config=VectorParams(size=EMBED_DIM, distance=Distance.COSINE),
)

# Embed all chunks in batches
BATCH_SIZE = 64
texts = [c.text for c in all_chunks]
all_embeddings = []

for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Embedding"):
    batch = texts[i : i + BATCH_SIZE]
    embs  = embed_model.encode(batch, normalize_embeddings=True)
    all_embeddings.extend(embs)

all_embeddings = np.array(all_embeddings)
ok(f"Embeddings shape: {all_embeddings.shape}")

# Upsert into Qdrant
points = [
    PointStruct(
        id=i,
        vector=all_embeddings[i].tolist(),
        payload={
            "chunk_id":  c.chunk_id,
            "doc_id":    c.doc_id,
            "title":     c.title,
            "text":      c.text,
            "full_text": c.full_text,
            "url":       c.url,
            "category":  c.category,
            "chunk_idx": c.chunk_idx,
        }
    )
    for i, c in enumerate(all_chunks)
]

# Upload in batches of 256
for i in tqdm(range(0, len(points), 256), desc="Uploading to Qdrant"):
    qdrant.upsert(collection_name=COLLECTION, points=points[i:i+256])

collection_info = qdrant.get_collection(COLLECTION)
ok(f"Qdrant collection '{COLLECTION}' ready — {collection_info.points_count} vectors indexed")

---
## 4 · Hybrid Search — BM25 + Dense + RRF Fusion

**Why not dense-only?**  
Dense retrieval is strong on semantic similarity. It misses exact-term matches — ticker symbols, product names, regulatory codes. BM25 nails exact matches but fails on paraphrase. Neither alone is sufficient for a support FAQ system where users ask both `"what is MTF"` (semantic) and `"MTF margin requirement"` (keyword).

**Reciprocal Rank Fusion (RRF):**  
Score = Σ 1/(k + rank_i) where k=60 is a smoothing constant. RRF is rank-based — it does not require score normalization between BM25 and cosine similarity, which makes it robust without hyperparameter tuning.

**Production trade-off:**  
BM25 runs on CPU in memory. Dense retrieval hits the vector index. For 100k+ chunks, the bottleneck shifts to BM25 RAM — at that scale, replace BM25 with Elasticsearch sparse vectors or Qdrant's built-in sparse support (FastEmbed).

In [ ]:
# Build BM25 index over tokenized chunk texts
hdr("Building BM25 sparse index")

def tokenize(text: str) -> List[str]:
    """Simple whitespace + lowercase tokenizer. Good enough for BM25."""
    return re.findall(r"[a-z0-9]+", text.lower())

tokenized_corpus = [tokenize(c.text) for c in all_chunks]
bm25_index = BM25Okapi(tokenized_corpus)

ok(f"BM25 index built over {len(tokenized_corpus)} chunks")
info("Average doc length (BM25): " + str(round(bm25_index.avgdl, 1)) + " tokens")

In [ ]:
def dense_search(query: str, top_k: int = 30) -> List[Tuple[int, float]]:
    """
    Query Qdrant for top_k nearest neighbours.
    Returns list of (point_index, cosine_score).
    """
    q_emb = embed_model.encode([query], normalize_embeddings=True)[0]
    results = qdrant.query_points(
        collection_name=COLLECTION,
        query=q_emb.tolist(),
        limit=top_k,
    )
    points = results[0] if isinstance(results, tuple) else results.points
    return [(r.id, r.score) for r in points]


def bm25_search(query: str, top_k: int = 30) -> List[Tuple[int, float]]:
    """
    BM25 retrieval. Returns list of (chunk_index, bm25_score).
    """
    q_tokens = tokenize(query)
    scores = bm25_index.get_scores(q_tokens)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [(int(idx), float(scores[idx])) for idx in top_indices if scores[idx] > 0]


def reciprocal_rank_fusion(
    dense_hits:  List[Tuple[int, float]],
    sparse_hits: List[Tuple[int, float]],
    k: int = 60,
    top_k: int = 20,
) -> List[Tuple[int, float]]:
    """
    Fuse two ranked lists using RRF.
    k=60 is the standard smoothing constant from the original RRF paper.
    """
    rrf_scores: Dict[int, float] = {}

    for rank, (idx, _) in enumerate(dense_hits):
        rrf_scores[idx] = rrf_scores.get(idx, 0) + 1.0 / (k + rank + 1)

    for rank, (idx, _) in enumerate(sparse_hits):
        rrf_scores[idx] = rrf_scores.get(idx, 0) + 1.0 / (k + rank + 1)

    fused = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    return fused[:top_k]


def hybrid_search(query: str, stage1_k: int = 30) -> List[Tuple[Chunk, float]]:
    """
    Full hybrid search pipeline.
    Returns list of (Chunk, rrf_score) sorted by descending RRF score.
    """
    dense_hits  = dense_search(query, top_k=stage1_k)
    sparse_hits = bm25_search(query, top_k=stage1_k)
    fused       = reciprocal_rank_fusion(dense_hits, sparse_hits, top_k=stage1_k)

    results = []
    for idx, score in fused:
        chunk = all_chunks[idx]
        results.append((chunk, score))
    return results


ok("Hybrid search functions ready.")

In [ ]:
# ── Demo: compare Dense-only vs Hybrid ───────────────────────────
hdr("Demo: Dense-only vs Hybrid Search")
DEMO_QUERY = "how do I withdraw funds from my Zerodha account"

print(f"Query: '{DEMO_QUERY}'\n")

dense_only  = dense_search(DEMO_QUERY, top_k=5)
hybrid_hits = hybrid_search(DEMO_QUERY, stage1_k=30)

print(f"{Fore.CYAN}{'─'*28} Dense-only top-5 {'─'*14}{Style.RESET_ALL}")
for rank, (idx, score) in enumerate(dense_only, 1):
    c = all_chunks[idx]
    print(f"  {rank}. [{score:.3f}] {c.title}")

print(f"\n{Fore.CYAN}{'─'*28} Hybrid top-5 (RRF) {'─'*12}{Style.RESET_ALL}")
for rank, (chunk, score) in enumerate(hybrid_hits[:5], 1):
    print(f"  {rank}. [{score:.4f}] {chunk.title}")

---
## 5 · Cross-Encoder Reranking

**The two-tower gap:** Bi-encoders (like MiniLM) encode query and document independently — they cannot model fine-grained token-level interactions between the two. This is fast but imprecise.

**Cross-encoders** process the (query, document) pair jointly through the full attention stack. They see every token of the query attending to every token of the document. Dramatically more accurate — but O(n) in inference cost.

**The production pattern:** Use bi-encoder to recall 20–30 candidates cheaply, then cross-encoder to rerank to final top-5. You pay cross-encoder cost only on the shortlist, not the full corpus.

**Latency budget:** With `ms-marco-MiniLM-L-6-v2` on CPU:  
- Bi-encoder recall on 500 chunks: ~80ms  
- Cross-encoder on 20 candidates: ~120ms  
- Total retrieval budget: ~200ms — well inside a 1.5s response SLA

In [ ]:
hdr("Loading Cross-Encoder reranker")
CE_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
cross_encoder = CrossEncoder(CE_MODEL_NAME)
ok(f"Cross-encoder: {CE_MODEL_NAME}")


def rerank(
    query: str,
    candidates: List[Tuple[Chunk, float]],
    final_k: int = 5,
) -> List[Tuple[Chunk, float]]:
    """
    Rerank hybrid search candidates with the cross-encoder.
    Input:  candidates from hybrid_search (shortlist of ~20–30)
    Output: top final_k chunks with cross-encoder relevance scores
    """
    if not candidates:
        return []

    pairs  = [(query, c.text) for c, _ in candidates]
    scores = cross_encoder.predict(pairs)

    ranked = sorted(
        zip([c for c, _ in candidates], scores),
        key=lambda x: x[1],
        reverse=True
    )
    return ranked[:final_k]


# ── Demo ─────────────────────────────────────────────────────────
t0 = time.time()
candidates = hybrid_search(DEMO_QUERY, stage1_k=25)
reranked   = rerank(DEMO_QUERY, candidates, final_k=5)
elapsed    = time.time() - t0

print(f"\nQuery: '{DEMO_QUERY}'")
print(f"Retrieval pipeline total: {elapsed*1000:.0f}ms\n")

print(f"{Fore.CYAN}{'─'*30} After cross-encoder reranking {'─'*1}{Style.RESET_ALL}")
for rank, (chunk, score) in enumerate(reranked, 1):
    print(f"  {rank}. [CE={score:.3f}] {chunk.title}")
    print(f"       {chunk.text[:120]}...")
    print()

---
## 6 · Adaptive-k Retrieval

**The fixed-k problem:** Setting `top_k=5` works well for narrow queries. For ambiguous queries — `"margin"` could mean MTF margin, intraday margin, or Options margin — you need more context. But padding every query with top-10 inflates prompt cost and dilutes the signal fed to the LLM.

**Adaptive-k** uses the distribution of cross-encoder scores to find the natural relevance cliff:

```
k = argmax( score_i − score_{i+1} )
```

The largest consecutive score drop is the boundary between relevant and irrelevant. Everything above the cliff goes into context. Everything below gets dropped — no LLM sees it, no token budget is consumed.

**When it fails:** Flat score curves with no clear cliff — usually when the query is highly ambiguous or the embedding space is too compressed. Mitigate with a minimum k of 2.

In [ ]:
def adaptive_k(
    ranked_chunks: List[Tuple[Chunk, float]],
    min_k: int = 2,
    max_k: int = 8,
) -> List[Tuple[Chunk, float]]:
    """
    Apply adaptive-k boundary detection on cross-encoder scored candidates.
    Returns only the chunks above the steepest score drop.

    k = argmax(score_i - score_{i+1})
    """
    if len(ranked_chunks) <= min_k:
        return ranked_chunks

    scores = [s for _, s in ranked_chunks[:max_k]]

    # Compute consecutive differences
    gaps = [scores[i] - scores[i+1] for i in range(len(scores)-1)]

    if not gaps:
        return ranked_chunks[:min_k]

    cliff_idx = int(np.argmax(gaps))   # index of steepest drop
    k = max(min_k, cliff_idx + 1)      # include everything above the cliff

    return ranked_chunks[:k]


# ── Demo: adaptive-k vs fixed-k on several query types ───────────
hdr("Demo: Adaptive-k vs Fixed-k")

test_queries = [
    "how to place a stop loss order",       # narrow — expect k=2 or 3
    "margin",                                # ambiguous — expect higher k
    "can I withdraw money on the same day",  # specific — expect k=2
]

for q in test_queries:
    candidates = hybrid_search(q, stage1_k=25)
    reranked   = rerank(q, candidates, final_k=8)
    adaptive   = adaptive_k(reranked, min_k=2, max_k=8)

    scores = [f"{s:.2f}" for _, s in reranked[:8]]
    gaps   = [
        f"{reranked[i][1]-reranked[i+1][1]:.2f}"
        for i in range(len(reranked)-1) if i < 7
    ]

    print(f"\n  Query: '{q}'")
    print(f"  CE scores: [{', '.join(scores)}]")
    print(f"  Gaps:      [{', '.join(gaps)}]")
    print(f"  {Fore.GREEN}Adaptive-k = {len(adaptive)}{Style.RESET_ALL}  "
          f"(Fixed-k=5 would use {min(5, len(reranked))} chunks)")
    for i, (chunk, score) in enumerate(adaptive, 1):
        print(f"    {i}. [CE={score:.3f}] {chunk.title}")

---
## 7 · Generation with Grounded Citations

We use `google/flan-t5-base` here — a small (250M param) instruction-tuned model that runs on CPU in Colab without any API key. It is not production-grade for generative quality, but it demonstrates the full pipeline architecture correctly.

**To swap in a better model:** Replace the generation cell with an OpenAI/Anthropic API call — the retrieval pipeline above is completely model-agnostic. The prompt template stays identical.

**Citation grounding:** Every answer cites the chunk IDs it used. If the model produces a claim not traceable to a retrieved chunk, that is your hallucination signal — catch it at evaluation time (Section 8).

In [ ]:
hdr("Loading generation model (Flan-T5-base — CPU, no API key)")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

GEN_MODEL     = "google/flan-t5-base"
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
gen_model     = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL)

def generator(prompt: str, max_new_tokens: int = 256, **kwargs) -> list:
    inputs  = gen_tokenizer(prompt, return_tensors="pt",
                            truncation=True, max_length=1024)
    outputs = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    text    = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return [{"generated_text": text}]

ok(f"Generator ready: {GEN_MODEL}")

In [ ]:
def build_prompt(query: str, chunks: List[Tuple[Chunk, float]]) -> str:
    """
    Construct a grounded RAG prompt.
    Each retrieved chunk is cited with a [SOURCE-N] tag.
    The model is instructed to only use provided context.
    """
    context_blocks = []
    for i, (chunk, score) in enumerate(chunks, 1):
        context_blocks.append(
            f"[SOURCE-{i}] {chunk.title}\n{chunk.text[:600]}"
        )
    context = "\n\n".join(context_blocks)

    prompt = f"""Answer the following question using ONLY the provided sources.
If the answer is not in the sources, say 'I could not find this in the Zerodha support docs.'
Cite sources as [SOURCE-N] inline.

SOURCES:
{context}

QUESTION: {query}

ANSWER:"""
    return prompt


@dataclass
class RAGResponse:
    query:    str
    answer:   str
    sources:  List[Chunk]
    latency_ms: float


def rag_query(
    query:      str,
    stage1_k:   int = 25,
    final_k:    int = 8,
    use_adaptive_k: bool = True,
) -> RAGResponse:
    """
    Full production RAG pipeline:
    Hybrid Search → Cross-Encoder Rerank → Adaptive-k → Generate
    """
    t0 = time.time()

    # Stage 1: Hybrid retrieval
    candidates = hybrid_search(query, stage1_k=stage1_k)

    # Stage 2: Cross-encoder reranking
    reranked = rerank(query, candidates, final_k=final_k)

    # Stage 3: Adaptive-k boundary detection
    if use_adaptive_k:
        final_chunks = adaptive_k(reranked, min_k=2, max_k=final_k)
    else:
        final_chunks = reranked[:5]

    # Stage 4: Generate grounded answer
    prompt = build_prompt(query, final_chunks)
    output = generator(prompt, do_sample=False)[0]["generated_text"]

    latency_ms = (time.time() - t0) * 1000

    return RAGResponse(
        query=query,
        answer=output,
        sources=[c for c, _ in final_chunks],
        latency_ms=latency_ms,
    )


ok("RAG pipeline assembled.")

In [ ]:
hdr("Running RAG Pipeline — Live Queries")

demo_queries = [
    "How do I withdraw money from Zerodha to my bank account?",
    "What is the difference between CNC and MIS orders?",
    "Why was my F&O trade rejected due to insufficient margin?",
    "How do I add a nominee to my Zerodha account?",
]

responses: List[RAGResponse] = []

for q in demo_queries:
    print(f"\n{Fore.YELLOW}Q: {q}{Style.RESET_ALL}")
    resp = rag_query(q)
    responses.append(resp)

    print(f"{Fore.GREEN}A: {resp.answer}{Style.RESET_ALL}")
    print(f"   Latency: {resp.latency_ms:.0f}ms | Sources used: {len(resp.sources)}")
    for i, src in enumerate(resp.sources, 1):
        print(f"   [{i}] {src.title} → {src.url}")

---
## 8 · Evaluation — Hit Rate, Faithfulness, Hallucination Detection

**The gap most teams skip:** A RAG system that looks good in demos can still fail silently in production. Two failure modes from the deck:

1. **Bad retrieval** — the right chunk was not retrieved at all. No amount of generation quality fixes this. Measure: Hit Rate @ k.
2. **Bad generation** — the chunk was retrieved but the model hallucinated or ignored it. Measure: Faithfulness score.

We use a **synthetic golden QA set** generated from the corpus itself — no human annotation required for a demo. In production, seed this with real user queries from your support ticket logs.

**Faithfulness check:** A simple heuristic — does the answer contain n-grams that appear in the retrieved source? Not a replacement for LLM-as-judge, but deterministic and fast.

In [ ]:
hdr("Building Synthetic Golden QA Set")

# Hand-crafted query → expected article title pairs
# In production: derive from support ticket logs or LLM-generated QA pairs
GOLDEN_QA = [
    {"query": "How do I withdraw money from Zerodha?",
     "expected_titles": ["fund withdrawal", "withdraw", "payout"]},

    {"query": "What is the difference between holdings and positions?",
     "expected_titles": ["holdings and positions", "difference between holdings"]},

    {"query": "What is MTF and how does it work?",
     "expected_titles": ["mtf", "margin trading facility"]},

    {"query": "What is F&O trading?",
     "expected_titles": ["futures and options", "f&o", "derivatives"]},

    {"query": "How to add a bank account in Zerodha?",
     "expected_titles": ["add bank", "bank account"]},

    {"query": "What are intraday margins?",
     "expected_titles": ["intraday", "margin", "mis"]},

    {"query": "How to create a Zerodha support ticket?",
     "expected_titles": ["ticket", "create a ticket", "raise a ticket"]},

    {"query": "What is a stop loss order?",
     "expected_titles": ["stop loss", "sl order"]},

    {"query": "How do I invest in mutual funds on Coin?",
     "expected_titles": ["mutual fund", "coin", "invest"]},

    {"query": "What is the settlement process for equity trades?",
     "expected_titles": ["settlement", "t+1", "delivery"]},
]

ok(f"Golden QA set: {len(GOLDEN_QA)} query-answer pairs")

In [ ]:
def hit_at_k(
    retrieved_chunks: List[Chunk],
    expected_title_fragments: List[str],
    k: int = 5,
) -> bool:
    """
    Returns True if any of the top-k retrieved chunks match
    at least one expected title fragment (case-insensitive substring).
    """
    top_titles = [c.title.lower() for c in retrieved_chunks[:k]]
    for frag in expected_title_fragments:
        for title in top_titles:
            if frag.lower() in title:
                return True
    return False


def faithfulness_score(answer: str, source_chunks: List[Chunk]) -> float:
    """
    Heuristic faithfulness check.
    Counts what fraction of answer trigrams appear in the source corpus.

    Not a replacement for LLM-as-judge — but deterministic, fast, and
    surprisingly effective at catching severe hallucinations.
    """
    def ngrams(text: str, n: int = 3) -> set:
        tokens = re.findall(r"[a-z0-9]+", text.lower())
        return {tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)}

    answer_ngrams = ngrams(answer)
    if not answer_ngrams:
        return 0.0

    source_text = " ".join(c.text for c in source_chunks)
    source_ngrams = ngrams(source_text)

    overlap = answer_ngrams & source_ngrams
    return len(overlap) / len(answer_ngrams)


ok("Evaluation functions defined.")

In [ ]:
hdr("Running Evaluation — Production RAG vs Naive Dense-Only")

# ── Production pipeline (Hybrid + Rerank + Adaptive-k) ────────────
prod_results = []
for qa in tqdm(GOLDEN_QA, desc="Production RAG"):
    candidates = hybrid_search(qa["query"], stage1_k=25)
    reranked   = rerank(qa["query"], candidates, final_k=8)
    final      = adaptive_k(reranked, min_k=2, max_k=8)
    hit        = hit_at_k([c for c, _ in final], qa["expected_titles"], k=5)
    prompt     = build_prompt(qa["query"], final)
    answer     = generator(prompt, do_sample=False)[0]["generated_text"]
    faith      = faithfulness_score(answer, [c for c, _ in final])
    prod_results.append({"hit": hit, "faithfulness": faith,
                         "k_used": len(final), "answer": answer})

# ── Naive baseline (dense-only, fixed top-5) ──────────────────────
naive_results = []
for qa in tqdm(GOLDEN_QA, desc="Naive Dense-only"):
    dense_hits = dense_search(qa["query"], top_k=5)
    chunks     = [all_chunks[idx] for idx, _ in dense_hits]
    hit        = hit_at_k(chunks, qa["expected_titles"], k=5)
    prompt     = build_prompt(qa["query"], [(c, 0.0) for c in chunks])
    answer     = generator(prompt, do_sample=False)[0]["generated_text"]
    faith      = faithfulness_score(answer, chunks)
    naive_results.append({"hit": hit, "faithfulness": faith,
                          "k_used": 5, "answer": answer})

In [ ]:
hdr("Evaluation Results")

prod_hit_rate = sum(r["hit"] for r in prod_results) / len(prod_results)
prod_faith    = sum(r["faithfulness"] for r in prod_results) / len(prod_results)
prod_avg_k    = sum(r["k_used"] for r in prod_results) / len(prod_results)

naive_hit_rate = sum(r["hit"] for r in naive_results) / len(naive_results)
naive_faith    = sum(r["faithfulness"] for r in naive_results) / len(naive_results)

print(f"{'Metric':<35} {'Naive Dense-only':>18} {'Production RAG':>18}")
print("─" * 73)
print(f"{'Hit Rate @ 5':<35} {naive_hit_rate:>17.1%} {prod_hit_rate:>17.1%}")
print(f"{'Faithfulness (trigram overlap)':<35} {naive_faith:>17.2f} {prod_faith:>17.2f}")
print(f"{'Avg chunks sent to LLM':<35} {'5 (fixed)':>18} {prod_avg_k:>17.1f}")
print("─" * 73)
print()

# Per-query breakdown
print(f"{'Query':<52} {'Naive Hit':>11} {'Prod Hit':>10} {'Prod k':>8}")
print("─" * 84)
for i, qa in enumerate(GOLDEN_QA):
    q = qa["query"][:50]
    nh = Fore.GREEN + "✓" + Style.RESET_ALL if naive_results[i]["hit"] else Fore.RED + "✗" + Style.RESET_ALL
    ph = Fore.GREEN + "✓" + Style.RESET_ALL if prod_results[i]["hit"]  else Fore.RED + "✗" + Style.RESET_ALL
    pk = prod_results[i]["k_used"]
    print(f"  {q:<60} {nh:>9} {ph:>19} {pk:>8}")

---
## 9 · Interactive Query Interface

Type any question about Zerodha below and the full production pipeline runs — Hybrid Search → Cross-Encoder Rerank → Adaptive-k → Grounded Generation.

In [ ]:
def ask(query: str, verbose: bool = True) -> RAGResponse:
    """
    Ask any question. Full pipeline runs automatically.
    Set verbose=False to suppress source details.
    """
    resp = rag_query(query)

    print(f"\n{Fore.YELLOW}{'─'*60}")
    print(f"Q: {query}")
    print(f"{'─'*60}{Style.RESET_ALL}")
    print(f"\n{Fore.GREEN}{resp.answer}{Style.RESET_ALL}")
    print(f"\n  Pipeline: Hybrid Search → CE Rerank → Adaptive-k={len(resp.sources)}")
    print(f"  Latency:  {resp.latency_ms:.0f}ms")

    if verbose:
        print(f"\n  {Fore.CYAN}Sources:{Style.RESET_ALL}")
        for i, src in enumerate(resp.sources, 1):
            print(f"    [{i}] {src.title}")
            print(f"         {src.url}")

    return resp


# ── Try it ──────────────────────────────────────────────────────
# Modify this query to test any Zerodha support question:
_ = ask("What documents do I need to open a Zerodha account?")

In [ ]:
# More queries — uncomment and run any of these
# print("Uncomment any query below and run this cell.")

_ = ask("How long does it take for a fund withdrawal to reach my bank?")
# _ = ask("What happens to my F&O position on expiry day?")
# _ = ask("Can I trade in NRI account on Zerodha?")
# _ = ask("What is a GTT order and how do I place one?")
# _ = ask("What is the brokerage charged on intraday equity trades?")
# _ = ask("How to convert CNC position to MIS?")

---
## Key Takeaways

```
Production RAG is not:
  Chunk → Embed → top-k → Generate

Production RAG is:
  Semantic Chunk (sentence-aware, overlapping)
  → Hybrid Retrieval (BM25 + Dense, fused with RRF)
  → Cross-Encoder Rerank (precision pass on shortlist)
  → Adaptive-k (score-gap boundary — no fixed k)
  → Grounded Generation (citations, no hallucination budget)
  → Continuous Eval (Hit Rate + Faithfulness per release)
```

**Swap out for production:**

| This notebook | Production equivalent |
|---------------|----------------------|
| `QdrantClient(':memory:')` | Qdrant Cloud / self-hosted |
| `flan-t5-base` | GPT-4o / Claude 3.5 / Gemini |
| `all-MiniLM-L6-v2` | Domain-fine-tuned embedding model |
| `BM25Okapi` in RAM | Elasticsearch sparse / Qdrant FastEmbed |
| Synthetic QA eval | Real user query logs + human annotation |

---
*Indranil Chandra · ML & Data Architect, Upstox · Co-organiser, GDG MAD Mumbai*